#### import libraries and define variables

In [ ]:
from trident_token_library_wrapper import PyTridentTokenLibrary
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests 

keyvault = 'https://xxxxx.vault.azure.net/'
tenantId_SecretName = 'xxxxx'
clientId_SecretName = 'xxxxxx'
clientSecret_SecretName = 'xxxx'

#### define functions to get key vault secrets, API bearer token and JSON responses

In [ ]:
def get_secret_from_keyvault(secret_name):
    access_token=PyTridentTokenLibrary.get_access_token("keyvault")
    return PyTridentTokenLibrary.get_secret_with_token(keyvault,secret_name,access_token)

def get_bearer_token():
    tenant_id = get_secret_from_keyvault(tenantId_SecretName)
    client_id = get_secret_from_keyvault(clientId_SecretName)
    client_secret = get_secret_from_keyvault(clientSecret_SecretName)
    url = "https://login.microsoftonline.com/" + tenant_id + "/oauth2/token"
    data = "grant_type=client_credentials&client_id=" + client_id + "&client_secret=" + client_secret + "&resource=https://analysis.windows.net/powerbi/api"  
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.post(url, headers=headers, data=data)
    return response.json()["access_token"]

def get_response_json(fullurl, method, data , payload_object):
    bearer_token = get_bearer_token()
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {bearer_token}'}
    response = requests.request(method, fullurl, headers=headers, data=data)
    return response.json()[payload_object]

#### get all capacities in tenant

In [ ]:
source = get_response_json('https://api.powerbi.com/v1.0/myorg/admin/capacities',"value")
df = spark.read.option("multiline","true").json(sc.parallelize(source))
df1 = df.drop('admins')
writetolake = df1.write.mode("overwrite").format("delta").save("Tables/Capacities")

#### get all capacity admins 

In [ ]:
source = get_response_json('https://api.powerbi.com/v1.0/myorg/admin/capacities',"GET", "", "value")
df = spark.read.option("multiline","true").json(sc.parallelize(source))
df1 = df.select(df.id, explode("admins").alias("admins"))
df1.show()
writetolake = df1.write.mode("overwrite").format("delta").save("Tables/CapacityAdmins")

#### query a dataset

In [ ]:
datasetId = 'xxxxxxxxxxxx'
DAXQuery = 'EVALUATE Date'
body = '{"queries":[{"query":"' + DAXQuery + '"}],"serializerSettings":{"incudeNulls":true}}'
source = get_response_json('https://api.powerbi.com/v1.0/myorg/datasets/' + datasetId + '/executeQueries',"POST", body, "results")
print(source)